In [98]:
# 建立資料庫
def creat_db():
    import sqlite3
    conn = sqlite3.connect('history_data.db')
    c = conn.cursor()

    c.execute('''DROP TABLE feature_history''')

    # Create table
    c.execute('''CREATE TABLE feature_history
                 (date text, type text ,upward real, downward real, sum real, delta real)''')

    # Insert a row of data
    # c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

    # Save (commit) the changes
    conn.commit()

    # We can also close the connection if we are done with it.
    # Just be sure any changes have been committed or they will be lost.
    conn.close()

In [37]:
# 測試date格式的for迴圈
from datetime import date
from dateutil.rrule import rrule, DAILY

a = date(2014, 1, 1)
b = date(2014, 1, 10)

for dt in rrule(DAILY, dtstart=a, until=b):
    print(dt.strftime("%Y-%m-%d"))
    
# 測試取得今天的日期
from datetime import datetime
datetime.today().date()

2014-01-01
2014-01-02
2014-01-03
2014-01-04
2014-01-05
2014-01-06
2014-01-07
2014-01-08
2014-01-09
2014-01-10


datetime.date(2017, 7, 10)

In [68]:
def insert_to_sqlite(data1,data2,data3):
    import sqlite3
    conn = sqlite3.connect('history_data.db')
    c = conn.cursor()
    data = [data1,data2,data3]
    c.executemany('INSERT INTO feature_history VALUES (?,?,?,?,?,?)', data)
    conn.commit()
    conn.close()

In [58]:
# 給header
def get_header():
    inputstr = """Accept:text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8
Accept-Encoding:gzip, deflate
Accept-Language:zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4,zh-CN;q=0.2,ja;q=0.2
Cache-Control:max-age=0
Connection:keep-alive
Content-Length:239
Content-Type:application/x-www-form-urlencoded
Cookie:ASPSESSIONIDCAACQTQB=BIGPDFPDPCPAOMHBIBDDPKCM; AX-cookie-POOL_PORTAL=AFACBAKM; AX-cookie-POOL_PORTAL_web3=ADACBAKM; ASPSESSIONIDACCASQQA=GHPJMAHAJIJGNIPAKKNGAEJI; ASPSESSIONIDACADQRQA=IOPCHONBALLMEKGLLKHFNKDF; ASPSESSIONIDCCDBRQQA=FGBDDKKCKBOMHGBAOKPIKKBO; ASPSESSIONIDQSTQDSTD=HLOCDCEAIKLPJNGKNMEDOLEB; ASPSESSIONIDACAARQRA=APEAPNABEDHDPEHFGCLDBLIO; ASPSESSIONIDQSRSCSTD=PMLFDLKBAPNEANNCABCBOIBF; ASPSESSIONIDAADBACQB=IHLODDNBAIJBFIPOAFIAKFAF; ASPSESSIONIDAAABDDQA=LPMLDGLDHOCOIBCGILAMIKGI; ASPSESSIONIDCCCDACRB=MIHICCIAIFNNIIEOMAHGFEAB; ASPSESSIONIDACDDBCRB=OPHLDPBBNBPDAFCKKEKLOEIK; ASPSESSIONIDCADAACQB=OCHOHNLAKEJLBMICPPLJMGAA
Host:www.taifex.com.tw
Origin:http://www.taifex.com.tw
Referer:http://www.taifex.com.tw/chinese/3/3_2_2.asp
Upgrade-Insecure-Requests:1
User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"""
    header = {}
    for each in inputstr.split('\n'):
        kv = each.split(':')
        key = kv[0]
        value = ':'.join(kv[1:]) 
        header[key] = value
    return header

In [52]:
# 把日期插入payload
def setdays(eachday):
    (year, month, day) = (str(eachday.year), str(eachday.month), str(eachday.day))
    inputstr = """goday:-1
COMMODITY_ID:"""
    payload = {}
    for each in inputstr.split('\n'):
        kv = each.split(':')
        key = kv[0]
        value = ':'.join(kv[1:]) 
        payload[key] = value
    payload['DATA_DATE_Y'] = year
    payload['DATA_DATE_M'] = month
    payload['DATA_DATE_D'] = day
    payload['syear'] = year
    payload['smonth'] = month
    payload['sday'] = day
    payload['datestart'] = year + '/' + month + '/' + day
    return payload

In [41]:
# 取得每日的data
def get_data(header,payload):
    import requests
    from bs4 import BeautifulSoup
    res = requests.post('http://www.taifex.com.tw/chinese/3/7_12_3.asp', headers=header ,data = payload)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text,'lxml')
    local = soup.select('#printhere > div')[3].select('table > tbody')[0].select('tr')[1].select('td > table > tbody > tr')[3].text.split()
    invester = soup.select('#printhere > div')[3].select('table > tbody')[0].select('tr')[1].select('td > table > tbody > tr')[4].text.split()
    foreign = soup.select('#printhere > div')[3].select('table > tbody')[0].select('tr')[1].select('td > table > tbody > tr')[5].text.split()
    local.remove(local[0])
    local.remove(local[0])
    return (local,invester,foreign)

In [99]:
# 主程式：設定開始跟結束日期，取得資料後存入資料庫
from datetime import date
from dateutil.rrule import rrule, DAILY
import sys

startdate = date(2014, 7, 11)
enddate = date(2017, 7, 10)
header = get_header()
previous = (0,0,0)
for dt in rrule(DAILY, dtstart=startdate, until=enddate):
    try:
        sys.stdout.write('\r' + str(dt))
        payload = setdays(dt)
        (local,invester,foreign) = get_data(header,payload)
        data1 = (str(dt.date()), 'local', local[7].replace(',',''),local[9].replace(',',''),local[11].replace(',',''),int(local[11].replace(',',''))-int(previous[0]))
        data2 = (str(dt.date()), 'invester', invester[7].replace(',',''),invester[9].replace(',',''),invester[11].replace(',',''),int(invester[11].replace(',',''))-int(previous[1]))
        data3 = (str(dt.date()), 'foreign', foreign[7].replace(',',''),foreign[9].replace(',',''),foreign[11].replace(',',''),int(foreign[11].replace(',',''))-int(previous[2]))
        previous = (local[11].replace(',',''),invester[11].replace(',',''),foreign[11].replace(',',''))
        insert_to_sqlite(data1,data2,data3)
    except IndexError:
        pass
#         print(dt)

2017-07-10 00:00:00